# Milestone 4: Modeling

 In our flight prices dataset, we will be creating a linear regression model to predict the total fare variable based on various features that are relevant to predicting a cost of a flight ticket.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import os

In [2]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("ML Pipeline") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.instances", "4") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# Google Cloud Storage path
gcs_path = 'gs://my-bigdatatech-project-jl/cleaned/Data_Cleaned.parquet'

# Read the Parquet file from GCS
df_spark = spark.read.parquet(gcs_path)

# Show the DataFrame schema and the first few rows
df_spark.printSchema()
df_spark.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 22:38:46 INFO SparkEnv: Registering MapOutputTracker
24/12/13 22:38:46 INFO SparkEnv: Registering BlockManagerMaster
24/12/13 22:38:46 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/13 22:38:46 INFO SparkEnv: Registering OutputCommitCoordinator


root
 |-- legId: string (nullable = true)
 |-- searchDate: date (nullable = true)
 |-- flightDate: date (nullable = true)
 |-- startingAirport: string (nullable = true)
 |-- destinationAirport: string (nullable = true)
 |-- fareBasisCode: string (nullable = true)
 |-- travelDuration: string (nullable = true)
 |-- elapsedDays: integer (nullable = true)
 |-- isBasicEconomy: boolean (nullable = true)
 |-- isRefundable: boolean (nullable = true)
 |-- isNonStop: boolean (nullable = true)
 |-- baseFare: double (nullable = true)
 |-- totalFare: double (nullable = true)
 |-- seatsRemaining: integer (nullable = true)
 |-- totalTravelDistance: integer (nullable = true)
 |-- segmentsDepartureTimeRaw: string (nullable = true)
 |-- segmentsArrivalTimeRaw: string (nullable = true)
 |-- segmentsArrivalAirportCode: string (nullable = true)
 |-- segmentsDepartureAirportCode: string (nullable = true)
 |-- segmentsAirlineName: string (nullable = true)
 |-- segmentsAirlineCode: string (nullable = true)
 |

+--------------------+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+------------------------+----------------------+--------------------------+----------------------------+--------------------+-------------------+----------------------------+-------------------------+----------------+-------------------+
|               legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode| segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDurationInSeconds|segmentsDistance|  segmentsCabinCode|
+--------------------+----------+----------+---------------+------------------+-------------+-------

In [3]:


# Filter the Spark DataFrame (example condition)
filtered_df = df_spark.filter(df_spark.totalFare.isNotNull() & df_spark.segmentsEquipmentDescription.isNotNull())

# Limit to 10 rows
limited_df = filtered_df.limit(50)

# Convert to Pandas DataFrame
pandas_df = limited_df.toPandas()

In [4]:


df_spark.dtypes

[('legId', 'string'),
 ('searchDate', 'date'),
 ('flightDate', 'date'),
 ('startingAirport', 'string'),
 ('destinationAirport', 'string'),
 ('fareBasisCode', 'string'),
 ('travelDuration', 'string'),
 ('elapsedDays', 'int'),
 ('isBasicEconomy', 'boolean'),
 ('isRefundable', 'boolean'),
 ('isNonStop', 'boolean'),
 ('baseFare', 'double'),
 ('totalFare', 'double'),
 ('seatsRemaining', 'int'),
 ('totalTravelDistance', 'int'),
 ('segmentsDepartureTimeRaw', 'string'),
 ('segmentsArrivalTimeRaw', 'string'),
 ('segmentsArrivalAirportCode', 'string'),
 ('segmentsDepartureAirportCode', 'string'),
 ('segmentsAirlineName', 'string'),
 ('segmentsAirlineCode', 'string'),
 ('segmentsEquipmentDescription', 'string'),
 ('segmentsDurationInSeconds', 'string'),
 ('segmentsDistance', 'string'),
 ('segmentsCabinCode', 'string')]

In [5]:


from pyspark.sql.functions import to_date

# convert date columns to actual date data type
df_spark = df_spark.withColumn('searchDate', to_date(df_spark.searchDate, 'yyyy-MM-dd'))
df_spark = df_spark.withColumn('flightDate', to_date(df_spark.flightDate, 'yyyy-MM-dd'))

In [6]:


from pyspark.sql.functions import date_format
from pyspark.sql.functions import col

# get year-month variable 
df_spark = df_spark.withColumn("searchdate_yearmonth", date_format(col("searchDate"), "yyyy-MM"))
df_spark = df_spark.withColumn('flightdate_yearmonth', date_format(col("flightDate"), "yyyy-MM"))

In [7]:


from pyspark.sql.functions import dayofweek

# get day of week as a number 
df_spark = df_spark.withColumn("searchDate_dayofweek", dayofweek(col("searchDate")) )
df_spark = df_spark.withColumn("flightDate_dayofweek", dayofweek(col("flightDate")) )

In [8]:


# get day of week as name e.g. Monday
df_spark = df_spark.withColumn("searchDate_dayofweekname", date_format(col("searchDate"), "E"))
df_spark = df_spark.withColumn("flightDate_dayofweekname", date_format(col("flightDate"), "E"))

In [9]:


from pyspark.sql.functions import when

# check if search or flight date falls on a weekend
df_spark = df_spark.withColumn("searchDate_weekend", when(df_spark.searchDate_dayofweek == 1, 1.0).when(df_spark.searchDate_dayofweek == 7, 1.0).otherwise(0))
# check if search or flight date falls on a weekend
df_spark = df_spark.withColumn("flightDate_weekend", when(df_spark.flightDate_dayofweek == 1, 1.0).when(df_spark.flightDate_dayofweek == 7, 1.0).otherwise(0))

In [10]:


from pyspark.sql.functions import col, when

# Define a list of holiday dates
holidays = [
    '2024-01-01',  # New Year's Day
    '2024-07-04',  # Independence Day
    '2024-12-25',  # Christmas Day
    '2024-11-28' # Thanksgiving Day
]

# Add a column to indicate if the flightDate is a holiday
df_spark = df_spark.withColumn(
    "isHoliday", 
    when(col("flightDate").cast("string").isin(holidays), 1).otherwise(0)
)

In [11]:


# Holiday Indicator
df_spark = df_spark.withColumn("isHoliday", when(col("flightDate").cast("string").isin(holidays), 1).otherwise(0))

In [12]:


from pyspark.sql.functions import month

# extract the month from flightDate and create a new column flightMonth
df_spark = df_spark.withColumn("flightMonth", month(col("flightDate")))

# create the season column based on the month
df_spark = df_spark.withColumn(
    "season",
    when(col("flightMonth").isin(6, 7, 8), "Summer")
    .when(col("flightMonth").isin(12, 1, 2), "Winter")
    .when(col("flightMonth").isin(3, 4, 5), "Spring")
    .otherwise("Fall")
)

In [13]:


from pyspark.sql.functions import lit, datediff

# 5. Flight Timing
df_spark = df_spark.withColumn("flightTiming", when((col("segmentsDepartureTimeRaw") >= lit("18:00:00")) &
                                            (col("segmentsArrivalTimeRaw") <= lit("06:00:00")), "Overnight")
                                       .otherwise("Daytime"))

# 6. Proximity of Booking
df_spark = df_spark.withColumn("daysUntilFlight", datediff("flightDate", "searchDate")) \
           .withColumn("bookingProximity", when(col("daysUntilFlight") <= 1, "Last Minute")
                                           .when((col("daysUntilFlight") > 1) & (col("daysUntilFlight") <= 7), "Within a Week")
                                           .otherwise("Planned in Advance"))

In [14]:


# check transformations
df_spark.select("flightTiming", "daysUntilFlight", "bookingProximity").show(10)

+------------+---------------+----------------+
|flightTiming|daysUntilFlight|bookingProximity|
+------------+---------------+----------------+
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
|     Daytime|              4|   Within a Week|
+------------+---------------+----------------+
only showing top 10 rows



In [15]:


from pyspark.sql.functions import col

# Converting boolean columns to integer - 0 or 1 - in PySpark
df_spark = df_spark.withColumn('isBasicEconomy', col('isBasicEconomy').cast('int'))
df_spark = df_spark.withColumn('isRefundable', col('isRefundable').cast('int'))
df_spark = df_spark.withColumn('isNonStop', col('isNonStop').cast('int'))

In [16]:


df_spark.select("isNonStop").show(10)

+---------+
|isNonStop|
+---------+
|        1|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
+---------+
only showing top 10 rows



In [17]:


# define seat availability volumes
df_spark = df_spark.withColumn(
    'seatAvailabilityCategory',
    when(col('seatsRemaining') <= 20, 'low')
    .when((col('seatsRemaining') > 20) & (col('seatsRemaining') <= 100), 'medium')
    .otherwise('high')
)

df_spark.select('seatsRemaining', 'seatAvailabilityCategory').show()

+--------------+------------------------+
|seatsRemaining|seatAvailabilityCategory|
+--------------+------------------------+
|             5|                     low|
|             2|                     low|
|             7|                     low|
|             3|                     low|
|             3|                     low|
|             4|                     low|
|             1|                     low|
|             4|                     low|
|             7|                     low|
|             7|                     low|
|             1|                     low|
|             1|                     low|
|             7|                     low|
|             7|                     low|
|             7|                     low|
|             7|                     low|
|             7|                     low|
|             7|                     low|
|             9|                     low|
|             9|                     low|
+--------------+------------------

In [18]:


from pyspark.sql.functions import col

# correct columns with right data types

df_spark = df_spark.withColumn("travelDuration", col("travelDuration").cast("float")) \
                   .withColumn("segmentsDurationInSeconds", col("segmentsDurationInSeconds").cast("int")) \
                   .withColumn("segmentsDistance", col("segmentsDistance").cast("int"))

In [19]:


from pyspark.sql.functions import col

# ensure columns are numeric
df_spark = df_spark.withColumn("elapsedDays", col("elapsedDays").cast("int")) \
           .withColumn("isBasicEconomy", col("isBasicEconomy").cast("int")) \
           .withColumn("isRefundable", col("isRefundable").cast("int")) \
           .withColumn("isNonStop", col("isNonStop").cast("int")) \
           .withColumn("baseFare", col("baseFare").cast("float")) \
           .withColumn("seatsRemaining", col("seatsRemaining").cast("int")) \
           .withColumn("totalTravelDistance", col("totalTravelDistance").cast("int")) \
           .withColumn("segmentsDurationInSeconds", col("segmentsDurationInSeconds").cast("int")) \
           .withColumn("segmentsDistance", col("segmentsDistance").cast("int"))

In [20]:
# Outlier detection for 'totalFare' using IQR
quantiles = df_spark.approxQuantile("totalFare", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_spark = df_spark.filter((col("totalFare") >= lower_bound) & (col("totalFare") <= upper_bound))

# Display summary statistics for verification
df_spark.select("totalFare").describe().show()

+-------+------------------+
|summary|         totalFare|
+-------+------------------+
|  count|          44007453|
|   mean|358.87227622574653|
| stddev|174.71544151851793|
|    min|             23.97|
|    max|            889.31|
+-------+------------------+



In [21]:


from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, FloatType, LongType

In [22]:


schema = StructType([
    StructField("legId", StringType(), True),
    StructField("searchDate", DateType(), True),
    StructField("flightDate", DateType(), True),
    StructField("startingAirport", StringType(), True),
    StructField("destinationAirport", StringType(), True),
    StructField("fareBasisCode", StringType(), True),
    StructField("travelDuration", StringType(), True),
    StructField("elapsedDays", IntegerType(), True),
    StructField("isBasicEconomy", IntegerType(), True),
    StructField("isRefundable", IntegerType(), True),
    StructField("isNonStop", IntegerType(), True),
    StructField("baseFare", FloatType(), True),
    StructField("totalFare", FloatType(), True),
    StructField("seatsRemaining", IntegerType(), True),
    StructField("totalTravelDistance", IntegerType(), True),
    StructField("segmentsDepartureTimeEpochSeconds", LongType(), True),
    StructField("segmentsArrivalTimeEpochSeconds", LongType(), True),
    StructField("segmentsDurationInSeconds", IntegerType(), True),
    StructField("segmentsDistance", IntegerType(), True),
    StructField("segmentsCabinCode", StringType(), True)
])

In [23]:


from pyspark.sql.functions import isnan

sampled_df = df_spark.sample(fraction=0.01, seed=42)

# split data into training and testing sets
train_data, test_data = sampled_df.randomSplit([0.7, 0.3], seed=42)

# remove nan values
train_data = train_data.filter(~isnan("totalFare"))
test_data = test_data.filter(~isnan("totalFare"))

# features for the model
feature_columns = ['isRefundable', 'isBasicEconomy', 'isNonStop', 'searchDate_weekend', 
                   'flightDate_weekend', 'isHoliday', 'season', 'flightTiming', 'bookingProximity']

feature_columns = [
    col for col in feature_columns if train_data.select(col).distinct().count() > 1
]

numeric_features = [col for col in feature_columns if train_data.schema[col].dataType.typeName() != 'string']

# index and encode categorical features
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "Index") for col in feature_columns if col not in numeric_features
]
encoders = [
    OneHotEncoder(inputCol=col + "Index", outputCol=col + "Vector") for col in feature_columns if col not in numeric_features
]

# assembling features into a vector
assembler = VectorAssembler(
    inputCols=[col + "Vector" for col in feature_columns if col not in numeric_features] + numeric_features,
    outputCol="features"
)

# linear Regression model
lr = LinearRegression(labelCol="totalFare")

# create Pipeline 
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

paramGrid = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.1, 0.01, 0.001]) 
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  
    .addGrid(lr.maxIter, [10, 50, 100])  
    .build()
)

evaluator = RegressionEvaluator(labelCol="totalFare", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(
    estimator=pipeline,  
    estimatorParamMaps=paramGrid,  
    evaluator=evaluator, 
    numFolds=3, 
    parallelism=2 
)

cv_model = crossval.fit(train_data)

# make predictions
predictions = cv_model.transform(test_data)

predictions = predictions.filter(~isnan("prediction") & ~isnan("totalFare"))

# evaluate the model
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})

print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")


predictions.select("flightDate", "totalFare", "prediction").show(10, truncate=False)

RMSE: 156.33667404473806
R-squared: 0.20038147143582818


+----------+---------+------------------+
|flightDate|totalFare|prediction        |
+----------+---------+------------------+
|2022-06-11|313.6    |438.92734152799164|
|2022-05-28|318.6    |424.9729059929551 |
|2022-07-03|388.7    |439.30342964076283|
|2022-06-06|172.6    |274.49387647382304|
|2022-07-01|521.59   |403.64870136218616|
|2022-06-04|526.21   |438.92734152799164|
|2022-05-15|308.6    |295.4419929918208 |
|2022-06-25|757.2    |438.92734152799164|
|2022-05-17|269.6    |389.6942658271496 |
|2022-06-18|172.6    |309.7725166396286 |
+----------+---------+------------------+
only showing top 10 rows



In [26]:


# transform the training data
transformed_train_data = cv_model.transform(train_data)

# save the transformed data to the /trusted folder
transformed_train_data.write.mode("overwrite").parquet("gs://my-bigdatatech-project-jl/trusted/processed_data")

print('transformed training data saved')

# save the LR model to the /models folder
cv_model.write().overwrite().save("gs://my-bigdatatech-project-jl/models/linear_regression_model")

print('LR model saved')

transformed training data saved


LR model saved


In [27]:
# Stop Spark session
spark.stop()